<a href="https://colab.research.google.com/github/MajiroZ/for_git_study/blob/master/Faster_R_CNN_YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##【問題1】学習と推定

論文 "Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks"
---

### 概要
Faster R-CNNは、物体検出のためのリアルタイムに近い性能を実現する統合ネットワークを提案しています。本手法では、Region Proposal Network (RPN) を導入し、検出ネットワークと畳み込み特徴を共有することで、高品質な領域提案をほぼコストゼロで生成します。RPNは、各位置で物体の境界と「物体らしさ」を同時に予測する完全畳み込みネットワークであり、Fast R-CNNと統合して1つのネットワークとして動作します。Pascal VOCやMS COCOなどのデータセットで、従来の最先端手法を上回る精度を実現し、5fps（GPU使用時）の速度を達成しました。

---

### 主な特徴と成果
1. **Region Proposal Network (RPN)**  
   RPNは物体の候補領域を生成する専用のネットワークで、畳み込み層を検出ネットワークと共有することで計算コストを削減しています。各スライディングウィンドウに対して複数のスケールとアスペクト比の「アンカー」を使用して予測を行います。

2. **高速化と精度の向上**  
   - 従来の領域提案手法（Selective Searchなど）より高速であり、計算時間を大幅に削減。
   - Pascal VOCでは平均適合率(mAP)が59.9%（300提案使用）を達成。
   - MS COCOデータセットでは、mAP@[.5, .95]で21.9%を記録。

3. **統合されたトレーニング**  
   RPNとFast R-CNNを交互に学習させることで、特徴量を効率的に共有し、統合された物体検出システムを構築。

4. **応用と拡張性**  
   Faster R-CNNは、3D物体検出やセグメンテーションなど、他のタスクにも適用可能で、ILSVRCやCOCO 2015競技会で1位を獲得しました。

---

### まとめ
Faster R-CNNは、物体検出の精度と速度を向上させるだけでなく、提案領域の品質も向上させました。この技術は、畳み込み層を共有することで、ほぼコストゼロで高品質な領域提案を可能にし、リアルタイムに近い物体検出を実現しました。

###README

In [ ]:
pip show keras

Name: keras
Version: 3.5.0
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [31]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import os
import glob

# 各フォルダのパスを指定
folder_paths = [
    "/content/drive/MyDrive/simpsons_dataset"
]

# すべての画像パスを収集
image_paths = []
for folder in folder_paths:
    image_paths.extend(glob.glob(os.path.join(folder, "*.jpg")))  # 画像の拡張子が .jpg の場合

# 確認
print(f"Total images found: {len(image_paths)}")
for path in image_paths[:5]:  # 最初の5つを表示
    print(path)


Mounted at /content/drive
Total images found: 0


In [32]:
cat /content/annotation.txt

ストリーミング出力は最後の 5000 行に切り捨てられました。
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg 57,72,52,72,9
./

In [33]:
file_path = "/content/simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg"

In [34]:
cp -r /content/drive/MyDrive/simpsons_dataset /content/

In [35]:
import cv2

test_image_path = "/content/simpsons_dataset/abraham_grampa_simpson/pic_0000.jpg"
img = cv2.imread(test_image_path)
if img is None:
    print(f"Failed to load image: {test_image_path}")
else:
    print(f"Image loaded successfully with shape: {img.shape}")

Image loaded successfully with shape: (416, 576, 3)


In [36]:
import os

with open("/content/annotation.txt", "r") as f:
    lines = f.readlines()

for line in lines:
    image_path = line.split(",")[0]  # 画像パスを抽出
    if not os.path.exists(image_path):
        print(f"Missing file: {image_path}")

In [37]:
from tensorflow.keras import initializers

In [38]:
bias_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01)

In [45]:
# Kerasを利用したFaster R-CNNの実装により物体検出を行う
from google.colab import drive
import os

# Check if the file exists
file_path = "/content/annotation.txt"

if os.path.exists(file_path):
    print("annotation.txt exists. Proceeding with training.")

    drive.mount('/content/drive')
    %cd /content

    !python /content/train.py -p annotation.txt
else:
    print("annotation.txt does not exist. Please upload the file or correct the path.")

annotation.txt exists. Proceeding with training.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
2025-01-14 08:59:46.958784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-14 08:59:46.998939: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-14 08:59:47.010682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 08:59:49.173542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Parsing annotation files
Training images per class (19

##【問題2】コードリーディング

### **1. RPN（Region Proposal Network）の実現箇所**
RPNは候補領域（アンカー）を生成し、スコアリングして物体のある可能性の高い場所を提案します。

#### 対応コード
```python
model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)
```
- `faster_rcnn.get_model()`がRPNモデルの生成と構成を担当します。

```python
loss_rpn = model_rpn.train_on_batch(X, Y)
P_rpn = model_rpn.predict_on_batch(X)
```
- `train_on_batch()`：RPNの損失（分類と回帰）を計算します。
- `predict_on_batch()`：画像内の各位置でアンカーのスコアと座標を予測します。

---

### **2. RoIプーリングの実現箇所**
RPNで生成されたRoI（領域）を固定サイズの特徴量に変換します。

#### 対応コード
```python
R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_data_format(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
X2, Y1, Y2 = roi_helpers.calc_iou(R, img_data, C, class_mapping)
```
- `roi_helpers.rpn_to_roi()`はRPNの出力を基に候補領域を生成します。
- `roi_helpers.calc_iou()`は候補領域とGround Truthの重なり（IoU）を計算します。

---

### **3. バックボーン（特徴抽出ネットワーク）の利用**
Faster R-CNNは事前学習済みモデル（VGG16など）をバックボーンとして利用します。

#### 対応コード
```python
model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)
```
この関数の内部で、特徴抽出に使用するモデルが定義されています。

---

### **4. アノテーションファイルの読み込み**
アノテーションデータを訓練用に解析します。

#### 対応コード
```python
all_imgs, classes_count, class_mapping = get_data(args.path)
```
- `get_data()`がアノテーションデータを読み込みます。

---

### **5. 訓練プロセス**
RPNと分類器の訓練がループ内で交互に行われます。

#### 対応コード
```python
for epoch_num in range(args.n_epochs):
    while True:
        X, Y, img_data = next(data_gen_train)
        loss_rpn = model_rpn.train_on_batch(X, Y)
        P_rpn = model_rpn.predict_on_batch(X)
        loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])
```

---

### **6. 最適なモデルの保存**
最小の損失を達成した時点でモデルを保存します。

#### 対応コード
```python
if curr_loss < best_loss:
    model_all.save_weights(C.model_path)
```

##【問題3】学習済みの重みによる推定

YOLOv3[2]のKeras実装を使います。

In [4]:
!wget https://pjreddie.com/media/files/yolov3.weights -O /content/yolov3.weights

--2025-01-14 06:56:38--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘/content/yolov3.weights’

/content/yolov3.wei 100%[===================>] 236.52M  14.9MB/s    in 16s     

2025-01-14 06:56:55 (14.5 MB/s) - ‘/content/yolov3.weights’ saved [248007048/248007048]



In [5]:
!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

2025-01-14 06:57:07.858125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-14 06:57:07.897410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-14 06:57:07.908432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 06:57:07.941129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 06:57:10.198986: W tensorflow/compiler/tf2

In [30]:
!python yolo_video.py --image

2025-01-14 08:36:53.467852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-14 08:36:53.493503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-14 08:36:53.500740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 08:36:54.948436: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Image detection mode
 Ignoring remaining command line arguments: ./path2your_video,
Anchors: [[ 10.  13.]
 [ 16.  30.]
 [ 33.  23.]
 [ 30.  61.]
 [ 62.  45.]
 [ 59. 119.]
 [116.  90.]
 [156. 198.]
 [373. 326.]]
num_classes: 80
Anchors count 

In [7]:
import os
print(os.path.exists('model_data/yolo.h5'))  # True なら OK

True


##【問題4】学習のためのファイルを作成

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [11]:
annotation_df = pd.read_csv('/content/annotation_rcnn.txt',header=None)

In [12]:
le = LabelEncoder()
annotation_df.iloc[:,5] = le.fit_transform(annotation_df.iloc[:,5])

In [13]:
rcnn_path = 'annotation_rcnn.txt'
yolo_path = 'annotation.txt'
n_sample, n_col = annotation_df.shape
with open(rcnn_path) as f:
    lines = f.readline()
    for i in range(n_sample):
        split_line = lines.split(',')
        image_path = split_line[0]
        split_line[0] = './' + image_path
        split_line[-1] = str(annotation_df.iloc[i,5]) + '\n'
        with open(yolo_path, mode='a') as out_f:
            join_line = ','.join(split_line)
            join_line = join_line.replace('.jpg,','.jpg ')
            out_f.write(join_line)

##【問題5】学習が行えることの確認

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/simpsons_dataset /content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!python train.py

ストリーミング出力は最後の 5000 行に切り捨てられました。
       [[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],


        [[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0.,